In [49]:
import os
import pandas as pd
import numpy as np
import shutil

In [4]:
# class numbers for bone loss
bonelossCLSS = [6]
# class numbers for dental caries
cariesCLSS = [2]

#directory for the labels
direct = 'C:\\Users\\rb01243\\OneDrive - University of Surrey\\Documents\\GitHub\\dental_disease\\notebooks\\datasets\\refined_dental_data\\labels'
direct2 = direct+'\\..'
try:
    os.mkdir(direct+'\\..'+'\\labels2')
    
    # use this for pre determined train test val format
    #os.mkdir(direct+'\\..'+'\\labels2'+'\\train')
    #os.mkdir(direct+'\\..'+'\\labels2'+'\\test')
    #os.mkdir(direct+'\\..'+'\\labels2'+'\\val')
except:
    print('directory already exists')

countBL, countDC = 0, 0
for root, subdirectories, files in os.walk(direct):
    #print(files)
    for text in files:
        f = open(root+ '\\' +text, "r")
        clss, x, y, w, h = [],[],[],[],[]
        for lines in f.readlines():
            spltLines = lines.split()
            spltItems = int(spltLines[0])
            if spltItems in bonelossCLSS:
                clss.append(0)
                countBL += 1 
            elif spltItems in cariesCLSS:
                clss.append(1)
                countDC += 1
            x.append(float(spltLines[1]))
            y.append(float(spltLines[2]))
            w.append(float(spltLines[3]))
            h.append(float(spltLines[4]))
        newLabels = pd.DataFrame({'label':clss, 'xCord':x, 'yCord':y, 'width':w, 'height':h})
        
        np.savetxt(direct+'\\..'+'\\labels2'+'\\'+text , newLabels.values, fmt='%i %1.10f %1.10f %1.10f %1.10f')
        # use this for pre determined train test val format instead of above line
        #np.savetxt(direct+'\\..'+'\\labels2'+'\\'+(root.split('\\')[-1])+'\\'+text , newLabels.values, fmt='%i %1.10f %1.10f %1.10f %1.10f')
        f.close()
        
print('Number of Bone Loss labels: ' + str(countBL))
print('Number of Dental Caries labels: ' + str(countDC))

Number of Bone Loss labels: 2097
Number of Dental Caries labels: 1219


## Run below to extract labels from the coco JSON file

In [75]:
# class numbers for bone loss
bonelossCLSS = [7]
# class numbers for dental caries
cariesCLSS = [3]
# class number for model
bLoss, caries = 0, 1

combLabels = cariesCLSS + bonelossCLSS

jsonFile = r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\dental_data_expert\via_export_coco.json'
imageFolder = r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\Data'
saveDirec = r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert'

try:
    os.mkdir(saveDirec+'\\expert')
except:
    print('parent folder already exists')
try:
    os.mkdir(saveDirec+'\\expert\\images')
    os.mkdir(saveDirec+'\\expert\\labels')
except:
    print('sub folders already exist')

with open(jsonFile, "r") as f:
    data = eval(f.read())

countBL, countDC = 0, 0
print(data.keys())
for image in data['images']:
    imgName = image['file_name']
    imgID = image['id']
    imgW = image['width']
    imgH = image['height']
    
    if imgName.split('.')[1] != 'JPG':
        continue
    
    annotations = data['annotations']
    annotations = [x for x in annotations if int(x['image_id']) == imgID]
    annotations = [x for x in annotations if int(x['category_id']) in combLabels]
    if not annotations:
        continue
    label, x, y, w, h = [],[],[],[],[]
    for anno in annotations:
        if int(anno['category_id']) in bonelossCLSS:
            label.append(bLoss)
        elif int(anno['category_id']) in cariesCLSS:
            label.append(caries)
        x1 = anno['bbox'][0]
        y1 = anno['bbox'][1]
        w1 = anno['bbox'][2]
        h1 = anno['bbox'][3]
        x1 = x1+(w1/2)
        y1 = y1+(h1/2)
        
        x.append(x1/imgW)
        y.append(y1/imgH)
        w.append(w1/imgW)
        h.append(h1/imgH)
        
    bBoxPerImg = pd.DataFrame({'label':label, 'xCord':x, 'yCord':y, 'width':w, 'height':h})
    txtLoc = saveDirec+'\\expert\\labels\\'+imgName.split('.')[0]+'.txt'
    print(txtLoc)
    np.savetxt(txtLoc, bBoxPerImg.values, fmt='%i %1.10f %1.10f %1.10f %1.10f')
    shutil.copyfile(imageFolder+'\\'+imgName, saveDirec+'\\expert\\images\\'+imgName)
print('######### DONE #########')



parent folder already exists
sub folders already exist
dict_keys(['info', 'images', 'annotations', 'licenses', 'categories'])
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191111-111458-XQ+M6AN4UCCT-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191111-111540-XE2TAIKRCCFW-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191112-081457-XOCFVWNUP+Q-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191112-084614-XGFT0CRMRV3T-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191112-095107-XNM0LLZFLZK-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expert\labels\Unknown-X-20191112-110314-XM1YNXPSERJC-3.txt
C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\expert\expe